# Préparation des données de défaillance

Chargement des donnnées

In [1]:
import pandas as pd
import numpy  as np

#Chargement des données
defaillance = pd.read_csv("../data/raw/PdM_failures.csv")
# transformer les dates dans le bon type
defaillance["datetime"] = pd.to_datetime(defaillance["datetime"])

#Création des indexes
defaillance = defaillance.set_index(['machineID', 'datetime'])
defaillance = pd.get_dummies(defaillance["failure"])
defaillance = defaillance.groupby(["machineID","datetime"]).sum()
defaillance["total_failure"]=defaillance.sum(axis = 1)

In [2]:
defaillance.head(8)

comp1  comp2  comp3  comp4  total_failure
machineID datetime                                                      
1         2015-01-05 06:00:00      0      0      0      1              1
          2015-03-06 06:00:00      1      0      0      0              1
          2015-04-20 06:00:00      0      1      0      0              1
          2015-06-19 06:00:00      0      0      0      1              1
          2015-09-02 06:00:00      0      0      0      1              1
          2015-10-17 06:00:00      0      1      0      0              1
          2015-12-16 06:00:00      0      0      0      1              1
2         2015-03-19 06:00:00      1      1      0      0              2

Toutes les maintenances sont enregistrées à 06:00H. On créée l'index pour chaque jour.

In [3]:
machines = pd.read_csv("../data/raw/PdM_machines.csv")

time_series = pd.date_range(start = "2015-01-01 06:00:00", end = "2016-01-01 06:00:00", freq="1D")
indexes = pd.MultiIndex.from_product([machines["machineID"].values,time_series.values], names=['machineID', 'datetime'])

On fait une jointure de ces indexes avec les données de maintenance

In [4]:
defaillance = pd.DataFrame(index = indexes).join(defaillance).fillna(0)

On va dupliquer les valeurs pour avoir un point toutes les 3h, annonçant une défaillance dans les 24h

In [5]:
time_series = pd.date_range(start = "2015-01-01 06:00:00", end = "2016-01-01 06:00:00", freq="3H")
indexes = pd.MultiIndex.from_product([machines["machineID"].values,time_series.values], names=['machineID', 'datetime'])
defaillance = pd.DataFrame(index = indexes).join(defaillance).fillna(method = 'bfill')
defaillance['failure_bool']=defaillance['total_failure']!=0
defaillance.head(20)

comp1  comp2  comp3  comp4  total_failure  \
machineID datetime                                                         
1         2015-01-01 06:00:00    0.0    0.0    0.0    0.0            0.0   
          2015-01-01 09:00:00    0.0    0.0    0.0    0.0            0.0   
          2015-01-01 12:00:00    0.0    0.0    0.0    0.0            0.0   
          2015-01-01 15:00:00    0.0    0.0    0.0    0.0            0.0   
          2015-01-01 18:00:00    0.0    0.0    0.0    0.0            0.0   
          2015-01-01 21:00:00    0.0    0.0    0.0    0.0            0.0   
          2015-01-02 00:00:00    0.0    0.0    0.0    0.0            0.0   
          2015-01-02 03:00:00    0.0    0.0    0.0    0.0            0.0   
          2015-01-02 06:00:00    0.0    0.0    0.0    0.0            0.0   
          2015-01-02 09:00:00    0.0    0.0    0.0    0.0            0.0   
          2015-01-02 12:00:00    0.0    0.0    0.0    0.0            0.0   
          2015-01-02 15:00:00    0.0    0.0    0.0    0.0            0.0   
          2015-01-02 18:00:00    0.0    0.0    0.0    0.0            0.0   
          2015-01-02 21:00:00    0.0    0.0    0.0    0.0            0.0   
          2015-01-03 00:00:00    0.0    0.0    0.0    0.0            0.0   
          2015-01-03 03:00:00    0.0    0.0    0.0    0.0            0.0   
          2015-01-03 06:00:00    0.0    0.0    0.0    0.0            0.0   
          2015-01-03 09:00:00    0.0    0.0    0.0    0.0            0.0   
          2015-01-03 12:00:00    0.0    0.0    0.0    0.0            0.0   
          2015-01-03 15:00:00    0.0    0.0    0.0    0.0            0.0   

                               failure_bool  
machineID datetime                           
1         2015-01-01 06:00:00         False  
          2015-01-01 09:00:00         False  
          2015-01-01 12:00:00         False  
          2015-01-01 15:00:00         False  
          2015-01-01 18:00:00         False  
          2015-01-01 21:00:00         False  
          2015-01-02 00:00:00         False  
          2015-01-02 03:00:00         False  
          2015-01-02 06:00:00         False  
          2015-01-02 09:00:00         False  
          2015-01-02 12:00:00         False  
          2015-01-02 15:00:00         False  
          2015-01-02 18:00:00         False  
          2015-01-02 21:00:00         False  
          2015-01-03 00:00:00         False  
          2015-01-03 03:00:00         False  
          2015-01-03 06:00:00         False  
          2015-01-03 09:00:00         False  
          2015-01-03 12:00:00         False  
          2015-01-03 15:00:00         False

In [6]:
defaillance.to_csv("../data/prepared_data/defaillance.csv")